<a href="https://colab.research.google.com/github/ba88052/rental_591_analize/blob/main/rental_591_analize_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Start

In [1]:
import pandas as pd
import scipy
import requests
import os
import re
import requests

def address_street(address):
    address = str(address)
    try:
        return(re.search(r"(?:區)(\D+)(?:\d?)", address).group(1))
    except:
        return(np.nan)
def get_street(df_for_clean):
    df_street = pd.DataFrame()
    df_street["street"] = df_for_clean["address"].apply(address_street)
    return df_street
def get_shape(df_for_clean):  
    df_for_clean = df_for_clean.rename(columns = {"shape":"shape_change"})
    shape_list = ['一年', '半年', '三個月', '住宅大樓', '公寓', '別墅', '華廈', '透天厝', '電梯大樓']
    df_shape =pd.DataFrame(columns = shape_list, index = df_for_clean.index)
    df_shape.fillna(0, inplace = True)
    for i,l in zip(df_for_clean.index, df_for_clean.shape_change):
        if l in shape_list:
            df_shape.loc[i, l] = 1
    return df_shape
def get_role(df_for_clean):    
    role_list = ['代理人', '仲介', '屋主']
    df_role =pd.DataFrame(columns = role_list, index = df_for_clean.index)
    df_role.fillna(0, inplace = True)
    for i,l in zip(df_for_clean.index, df_for_clean.role):
        df_role.loc[i, l] = 1
    return df_role
def get_kind(df_for_clean):    
    kind_list = ['其他', '分租套房', '整層住家', '獨立套房', '車位', '雅房']
    df_kind =pd.DataFrame(columns = kind_list, index = df_for_clean.index)
    df_kind.fillna(0, inplace = True)
    for i,l in zip(df_for_clean.index, df_for_clean.kind):
        df_kind.loc[i, l] = 1
    return df_kind
def get_section(df_for_clean):
    section_list = ['西屯區','北區','北屯區','西區', '南區','南屯區','東區','大里區','龍井區','太平區',
                    '沙鹿區','中區','大雅區','豐原區','霧峰區','潭子區','后里區','清水區','梧棲區','烏日區',
                    '大甲區','大肚區','神岡區','外埔區','東勢區','新社區','大安區','石岡區','和平區']
    df_section =pd.DataFrame(columns = section_list, index = df_for_clean.index)
    df_section.fillna(0, inplace = True)
    for i,l in zip(df_for_clean.index, df_for_clean.section):
        df_section.loc[i, l] = 1
    return df_section
def get_layout_spilt(df_for_clean):
    bedroom_list = []
    livingroom_list = []
    bathroom_list = []
    for i, l in zip(df_for_clean["layout"], df_for_clean["shape"]):
        try:
            bedroom_list.append(re.search(r"(\d+)房+", i).group(1))
        except:
            if l in ['住宅大樓', '公寓', '別墅', '華廈', '透天厝', '電梯大樓']:
                bedroom_list.append(1)
            else:
                bedroom_list.append(0)
        try:
            livingroom_list.append(re.search(r"(\d+)廳+", i).group(1))
        except:
            livingroom_list.append(0)
        try:
            bathroom_list.append(re.search(r"(\d+)衛+", i).group(1))
        except:
            bathroom_list.append(0)
    df_layout = pd.DataFrame()
    df_layout["bedroom"] = bedroom_list
    df_layout["livingroom"] = livingroom_list
    df_layout["bathroom"] = bathroom_list
    return df_layout

def get_rule_spilt(df_for_clean):
    girl_cant_live_list = []
    boy_cant_live_list = []
    pet_cant_live_list = []
    cant_cooking_list = []
    # boy_cant_live
    for i in df_for_clean["rule"]:
        i = str(i)
        if i:
            if "限男" in i:
                girl_cant_live_list.append(1)
            else:
                girl_cant_live_list.append(0)

            if "限女" in i:
                boy_cant_live_list.append(1)
            else:
                boy_cant_live_list.append(0)

            if "不可養寵物" in i:
                pet_cant_live_list.append(1)
            else:
                pet_cant_live_list.append(0)

            if "不可開伙" in i:
                cant_cooking_list.append(1)
            else:
                cant_cooking_list.append(0)
        else:
            girl_cant_live_list.append(0)
            boy_cant_live_list.append(0)
            pet_cant_live_list.append(0)
            cant_cooking_list.append(0)
    df_rule = pd.DataFrame()
    df_rule["girl_cant_live"] = girl_cant_live_list
    df_rule["boy_cant_live"] = boy_cant_live_list
    df_rule["pet_cant_live"] = pet_cant_live_list
    df_rule["cant_cooking"] = cant_cooking_list
    return df_rule

def get_clean_df(df_for_clean):
    df_clean = df_for_clean.copy()
    try:
        df_clean.drop(["Unnamed: 0"], axis = 1, inplace = True)
    except:
        None
    df_clean.drop(df_clean[df_clean["price"].isnull()].index, inplace = True)
    df_clean = pd.concat([df_clean, get_street(df_clean),
                        get_layout_spilt(df_clean), get_rule_spilt(df_clean), 
                        get_section(df_clean), get_kind(df_clean), 
                        get_shape(df_clean), get_role(df_clean)], axis = 1)
    df_clean.drop(["section", "inName", "phone", "mobile", 
                "phone_extension", "mobile_extension", 
                "community", "layout", "address", 
                "rule", "shape", "role", "kind"], axis = 1, inplace = True)
    return df_clean

#spider 591

In [2]:
#spider-591

#API網址    
url = "http://35.229.148.113:8081/spider-591"

#標頭
headers = {"Content-Type": "application/json"}

#參數
data = {
        "showMore": "1","searchtype": "1",
        "region": "8","section":"118",
        "multiPrice": "","rentprice": "",
        "kind": "","shape": "",
        "multiNotice": "","multiRoom": "",
        "multiArea": "","area": "",
        "multiFloor": "","option": ""
    }

#轉json並存成DataFrame
access_token = requests.post(url, headers=headers, json=data)
df_demo = pd.DataFrame.from_dict(access_token.json())
df_demo.head()

,post_id,title,kind,community,area,section,shape,layout,address,inName,role,phone,phone_extension,mobile,mobile_extension,rule,remark,price
13531712,13531712,免管理費全新全配備電梯套房,分租套房,,8,神岡區,透天厝,,台中市神岡區社南街5巷,陳先生,仲介,None,None,0933552280,None,此房屋男女皆可租住，不可養寵物，不可開伙,社口麥當勞旁全新套房&nbsp;&nbsp;電梯&nbsp;獨洗獨曬精裝全配備&nbsp;:...,8600
13486977,13486977,優質寧靜、精美裝潢、交通便利、整潔舒適,分租套房,,6,神岡區,透天厝,,台中市神岡區大裡街29巷16號,陳啟邦,代理人,None,None,0911769500,None,此房屋男女皆可租住，不可養寵物，不可開伙；適合上班族,＊＊＊＊＊乾淨優質、環境好、房客單純、寧靜＊＊＊＊＊1.自租免仲介費，免管理費.清潔費.公用...,6800
13497432,13497432,豐原交流道旁精美全配套房,分租套房,,6,神岡區,透天厝,,台中市神岡區大漢街31巷,楊家綾,仲介,0425123609,None,0952635497,None,此房屋男女皆可租住，不可養寵物，不可開伙,◎優質精美套房出租◎家電家具齊全，一卡皮箱即可入住◎採光良好通風佳◎近交流道，交通便捷◎附近...,6000
13535810,13535810,與學校圖書館當鄰居歡迎上班族租任,獨立套房,,6,神岡區,透天厝,--,台中市神岡區中山路,陳女士,屋主,None,None,0910584345,None,此房屋男女皆可租住，不可養寵物，不可開伙；適合學生及上班族,1.環境清幽、生活機能便利、進出有管制2.有獨立對外窗、採光良好、通風性佳3.緊鄰國小、圖書...,6000
13480253,13480253,近速聯綠點公司優質出租包水電,雅房,,6,神岡區,透天厝,,台中市神岡區中山路1584-7巷2號,王先生,屋主,0425623072,None,0971789225,None,此房屋限男生租住，不可養寵物，不可開伙；適合學生及上班族,如題：神岡區/雅房/全新裝潢/包水電管理費清潔費(冷氣費除外/一度5元)房間一：2F 雅房&...,4800


#price predict model

In [3]:
#training data
df_train = get_clean_df(df_demo.reset_index(drop = True))
df_train = df_train.drop(["street", "title", "remark", "post_id"], axis = 1)
df_train

,area,price,bedroom,livingroom,bathroom,girl_cant_live,boy_cant_live,pet_cant_live,cant_cooking,西屯區,...,三個月,住宅大樓,公寓,別墅,華廈,透天厝,電梯大樓,代理人,仲介,屋主
0,8,8600,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,1,0
1,6,6800,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
2,6,6000,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,1,0
3,6,6000,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
4,6,4800,1,0,0,1,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
5,39,27000,3,2,2,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
6,33.5,17000,3,1,2,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
7,7,6300,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
8,12,7000,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
9,8,6500,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,1,0


In [8]:
#price_predict_model
#API網址    
url = "http://35.229.148.113:8081/model"

#標頭
headers = {"Content-Type": "application/json"}

#參數
data = {
    "model":"XGB",
    "area": "35.7","bedroom": "4",
    "livingroom": "2","bathroom": "2",
    "section": "霧峰區", "kind": "整層住家",
    "shape": "電梯大樓","role": "仲介", 
    "girl_cant_live": 0,"boy_cant_live": 1,
    "pet_cant_live": 1,"cant_cooking":0
}

access_token = requests.post(url, headers=headers, json=data)
print("價格：",access_token.text)

價格： 122390.95


In [12]:
data = [{"model":"XGB","area": "35.7",
        "bedroom": "4","livingroom": "2",
        "bathroom": "2","section": "霧峰區", 
        "kind": "整層住家","shape": "電梯大樓",
        "role": "仲介", "girl_cant_live": 0,
        "boy_cant_live": 1,"pet_cant_live": 1,
        "cant_cooking":0},
        {"model":"XGB","area": "40",
        "bedroom": "4","livingroom": "2",
        "bathroom": "2","section": "南區", 
        "kind": "整層住家","shape": "電梯大樓",
        "role": "仲介", "girl_cant_live": 1,
        "boy_cant_live": 0,"pet_cant_live": 0,
        "cant_cooking":0},
        {"model":"keras","area": "10",
        "bedroom": "4","livingroom": "2",
        "bathroom": "2","section": "南屯區", 
        "kind": "整層住家","shape": "電梯大樓",
        "role": "仲介", "girl_cant_live": 0,
        "boy_cant_live": 0,"pet_cant_live": 0,
        "cant_cooking":0}]

In [13]:
for i in data:
    access_token = requests.post(url, headers=headers, json=i)
    print("價格：", access_token.text)

價格： 122390.95
價格： 116681.734
價格： 310919.16


Model in Web: http://35.229.148.113:8081/model/web

#data platform

Data Platform: http://35.229.148.113:8081/dash/